In [75]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import tenseal as ts



In [76]:
# Load the heart disease data
heart_data = pd.read_csv('heart_disease_data.csv')

# Preprocessing
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

scaler = StandardScaler()
scaler.fit(X)
standardized_data = scaler.transform(X)
X = standardized_data

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, stratify=Y, random_state=2)

# Train the model
model = svm.SVC(kernel='linear')
model.fit(X_train, Y_train)

SVC(kernel='linear')

In [77]:
# Perform predictions on the training set
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score on training data: ', training_data_accuracy)


Accuracy score on training data:  0.856353591160221


In [78]:

# Perform predictions on the testing set
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score on testing data: ', test_data_accuracy)


Accuracy score on testing data:  0.819672131147541


In [79]:

# Now, let's perform homomorphic encryption on the model and data
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = 2**40
context.generate_galois_keys()

# Encrypt the test data
enc_X_test = [ts.ckks_vector(context, X_test[i, :].tolist()) for i in range(len(X_test))]



In [80]:

# Encrypt the model coefficients
enc_model_coef = ts.ckks_vector(context, model.coef_[0].tolist())

In [81]:
# Perform encrypted prediction
enc_predictions = [enc_X_test[i].dot(enc_model_coef) for i in range(len(X_test))]


In [82]:
# Decrypt the predictions
decrypted_predictions = [vec.decrypt() for vec in enc_predictions]

In [83]:
# Flatten the list of lists obtained from decryption
flat_decrypted_predictions = [item for sublist in decrypted_predictions for item in sublist]

# Set a threshold for classification
threshold = 0.50

# Convert continuous predictions to binary (0 or 1)
binary_predictions = [1 if pred >= threshold else 0 for pred in flat_decrypted_predictions]

# Evaluate accuracy on encrypted predictions
encrypted_test_data_accuracy = accuracy_score(binary_predictions, Y_test)
print('Accuracy score on encrypted testing data: ', encrypted_test_data_accuracy)


Accuracy score on encrypted testing data:  0.7950819672131147


In [84]:
# Encrypt the input data (a vector of features)
input_data = np.array([44,0,2,108,141,0,1,175,0,0.6,1,0,2])
enc_input_data = ts.ckks_vector(context, input_data.tolist())


In [85]:
print(enc_input_data)

In [86]:


# Perform encrypted prediction
enc_prediction = enc_input_data.dot(enc_model_coef)

# Decrypt the prediction
decrypted_prediction = enc_prediction.decrypt()

print(decrypted_prediction)

[39.894015959531295]
